<a href="https://colab.research.google.com/github/b62mp1/files/blob/main/Graphs%26Metrics/Dates/2015-01-01/LSTM_2015-01-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Activation, concatenate, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go
import plotly

In [11]:
stock_name = "GOOG"
dataset=pdr.get_data_yahoo(stock_name, start='2015-01-01', end='2022-04-19').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (1836, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-01-02,2015-01-02,527.561584,529.815369,522.665039,523.373108,523.373108,1447563
2015-01-05,2015-01-05,521.827332,522.894409,511.655243,512.463013,512.463013,2059840
2015-01-06,2015-01-06,513.589966,514.761719,499.678131,500.585632,500.585632,2899940
2015-01-07,2015-01-07,505.611847,505.855164,498.281952,499.727997,499.727997,2065054
2015-01-08,2015-01-08,496.626526,502.101471,489.655640,501.303680,501.303680,3353582


In [12]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

scaler = MinMaxScaler(feature_range=(0, 1))
close_data = scaler.fit_transform(close_data)

In [13]:
# Splitting data into training and testing sets
split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = dataset['Date'][:split]
date_test = dataset['Date'][split:]

In [14]:
look_back = 5

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)

In [15]:
def predict(num_prediction, model):
    prediction_list = close_test[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = dataset['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

In [16]:
# Clear any back end stored data due to multiple iterations
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# Building LSTM model
model = Sequential()
model.add(
    LSTM(1000,
        activation='relu',
        input_shape=(look_back,1)
        )
)
model.add(Dense(1))
model.compile(optimizer="adam", loss='mse')

num_epochs = 25
model.fit(train_generator,epochs=num_epochs, verbose=1)

Epoch 1/25
74/74 [==============================] - 14s 172ms/step - loss: 0.0052
Epoch 2/25
74/74 [==============================] - 15s 198ms/step - loss: 1.4900e-04
Epoch 3/25
74/74 [==============================] - 10s 134ms/step - loss: 2.0100e-04
Epoch 4/25
74/74 [==============================] - 10s 134ms/step - loss: 4.6955e-04
Epoch 5/25
74/74 [==============================] - 10s 137ms/step - loss: 2.0146e-04
Epoch 6/25
74/74 [==============================] - 10s 137ms/step - loss: 2.2221e-04
Epoch 7/25
74/74 [==============================] - 10s 136ms/step - loss: 3.8719e-04
Epoch 8/25
74/74 [==============================] - 10s 136ms/step - loss: 1.8622e-04
Epoch 9/25
74/74 [==============================] - 10s 136ms/step - loss: 2.3576e-04
Epoch 10/25
74/74 [==============================] - 12s 158ms/step - loss: 2.7082e-04
Epoch 11/25
74/74 [==============================] - 10s 137ms/step - loss: 2.2346e-04
Epoch 12/25
74/74 [==============================] - 10s

In [17]:
# Predict and plot the results
prediction = model.predict(test_generator)

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

close_train = scaler.inverse_transform(close_train.reshape(-1, 1)).reshape((-1))
close_test = scaler.inverse_transform(close_test.reshape(-1, 1)).reshape((-1))
prediction = scaler.inverse_transform(prediction).reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data',
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Predicted Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Test Data'
)
layout = go.Layout(
    title = stock_name,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()